# Lesson 12 - Advanced SQL Window Functions

## ` PARTITION` and `OVER`

Window functions uses `PARTITION BY` and `OVER`. They allow a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row - the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result. 

**Note that you cannot use window functions and standard aggregations in the same query. More specifically, you can't include window functions in a `GROUP BY` clause. The `PARTITION BY` clause sets the range of records that will be used for each `GROUP` within the `OVER` clause.**

Useful resources: 
- https://www.postgresql.org/docs/9.1/static/tutorial-window.html
- https://blog.sqlauthority.com/2015/11/04/sql-server-what-is-the-over-clause-notes-from-the-field-101/
- https://www.postgresql.org/docs/8.4/static/functions-window.html
- https://stackoverflow.com/questions/561836/oracle-partition-by-keyword



<img src="../SQL/ERD DAND.jpg" width="600" height="400">

**Examples: Creating a Running Total Using Window Functions**

Using Derek's previous video as an example, create another running total. This time, create a running total of standard_amt_usd (in the orders table) over order time with no date truncation. Your final table should have two columns: one with the amount being added for each new row, and a second with the running total.

`SELECT standard_amt_usd,
SUM(standard_amt_usd) OVER (ORDER BY occurred_at) AS running_total
FROM orders`



**Examples: Creating a Partitioned Running Total Using Window Functions**
Now, modify your query from the previous quiz to include partitions. Still create a running total of standard_amt_usd (in the orders table) over order time, but this time, date truncate occurred_at by year and partition by that same year-truncated occurred_at variable. Your final table should have three columns: One with the amount being added for each row, one for the truncated date, and a final columns with the running total within each year.

`SELECT standard_amt_usd,
       DATE_TRUNC('year', occurred_at) as year,
       SUM(standard_amt_usd) OVER (PARTITION BY DATE_TRUNC('year', occurred_at) ORDER BY occurred_at) AS running_total
FROM orders`




## `ROW_NUMBER()`

This assigns a row number. Frequently used with `PARTITION BY` for determing the basis of row assignment. 


## `RANK()` and `DENSE_RANK()`

This is just like `ROW_NUMBER()` except for rows with equal values, it will skip values and assign the same value to each occurrance. `DENSE_RANK()` is the same `RANK()` except it won't skip values. For example, if rows 3,4,5 are all the same, then `RANK` will assign a row number of 5 for each of those rows. However, `DENSE_RANK()` will assign a value of 3 for each of them.

**Example: Ranking Total Paper Ordered by Account**
Select the id, account_id, and total variable from the orders table, then create a column called total_rank that ranks this total amount of paper ordered (from highest to lowest) for each account using a partition. Your final table should have these four columns.

`SELECT o.id, o.account_id, o.total, a.name,
RANK() OVER (PARTITION BY a.name ORDER BY o.total DESC) AS total_rank
FROM orders o
JOIN accounts a
ON a.id = o.account_id`

*OR*

`SELECT id,
       account_id,
       total,
       RANK() OVER (PARTITION BY account_id ORDER BY total DESC) AS total_rank
FROM orders`




**A more complex example:**

`SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders`


## Aggregates in Window Functions with and without `ORDER BY`
The `ORDER BY` clause is one of two clauses integral to window functions. The **ORDER** and **PARTITION** define what is referred to as the “window”—the ordered subset of data over which calculations are made. Removing `ORDER BY` just leaves an unordered partition; in our query's case, each column's value is simply an aggregation (e.g. sum, count, average, minimum, or maximum) of all the standard_qty values in its respective account_id.

As Stack Overflow user mathguy explains:

The easiest way to think about this - leaving the `ORDER BY` out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other. Indeed, you can get the same effect by explicitly adding the `ORDER BY` clause like this: `ORDER BY` 0 (or "order by" any constant expression), or even, more emphatically, `ORDER BY NULL`.




## Using Alias Function

This is made up of two parts: the Alias (uses `WINDOW <name> AS expression`) between the `WHERE` clause and the `GROUP BY` clause;  and the Windows function part (normally with `PARTITION` statement). Here's an example:

`SELECT id,
       account_id,
       DATE_TRUNC('year',occurred_at) AS year,
       DENSE_RANK() OVER account_year_window AS dense_rank,
       total_amt_usd,
       SUM(total_amt_usd) OVER account_year_window AS sum_total_amt_usd,
       COUNT(total_amt_usd) OVER account_year_window AS count_total_amt_usd,
       AVG(total_amt_usd) OVER account_year_window AS avg_total_amt_usd,
       MIN(total_amt_usd) OVER account_year_window AS min_total_amt_usd,
       MAX(total_amt_usd) OVER account_year_window AS max_total_amt_usd
FROM orders 
WINDOW account_year_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))`

## Comparing Rows


## `LAG` 

Used to compare data in the preceding row. Use `LAG(column_name)`


## `LEAD`

Used to compare data in the following row. Use `LEAD(column_name)`.

**Example: Imagine you're an analyst at Parch & Posey and you want to determine how the current order's total revenue (i.e. from sales of all types of paper) revenue compares to the next order's total revenue.**

`SELECT occurred_at,
       total_sales,
       LEAD(total_sales) OVER (ORDER BY occurred_at) AS lead,
       LEAD(total_sales) OVER (ORDER BY occurred_at) - total_sales AS lead_difference
FROM (
SELECT occurred_at, SUM(total_amt_usd) AS total_sales
  FROM orders 
 GROUP BY 1
 ) sub`


<img src="../SQL/lead_example.png" width="600" height="400">


## `NTILE`

This identifies what percentile (or quartile, or any other subdivision) a given row falls into. The syntax is `NTILE(*# of budgets*)`. `ORDER BY` is often used to determine which column to use to determine the quartiles (or whatever number of tiles you specify). Example: `NTILE(4) OVER (ORDER BY standard_qty) AS quartile` or `NTILE(100) OVER (ORDER BY standard_qty) AS percentile`

Note that this doesn't work as well for relatively few rows as there isn't enough data for a meaningful spread.

## Example: Percentiles with Partitions

You can use partitions with percentiles to determine the percentile of a specific subset of all rows. Imagine you're an analyst at Parch & Posey and you want to determine the largest orders (in terms of quantity) a specific customer has made to encourage them to order more similarly sized large orders. You only want to consider the NTILE for that customer's account_id.

In the SQL Explorer below, write three queries (separately) that reflects each of the following:

Use the NTILE functionality to divide the accounts into 4 levels in terms of the amount of standard_qty for their orders. Your resulting table should have the account_id, the occurred_at time for each order, the total amount of standard_qty paper purchased, and one of four levels in a standard_quartile column.

`SELECT id,
       account_id,
       occurred_at,
       standard_qty,
       NTILE(4) OVER (PARTITION BY account_id ORDER BY standard_qty) AS standard_quartile
FROM orders 
ORDER BY account_id`



Use the NTILE functionality to divide the accounts into two levels in terms of the amount of gloss_qty for their orders. Your resulting table should have the account_id, the occurred_at time for each order, the total amount of gloss_qty paper purchased, and one of two levels in a gloss_half column.

`SELECT id,
       account_id,
       occurred_at,
       gloss_qty,
       NTILE(2) OVER (PARTITION BY account_id ORDER BY gloss_qty) AS gloss_half 
FROM orders
ORDER BY account_id`


Use the NTILE functionality to divide the accounts into 100 levels in terms of the amount of total_amt_usd for their orders. Your resulting table should have the account_id, the occurred_at time for each order, the total amount of total_amt_usd paper purchased, and one of 100 levels in a total_percentile column.


`SELECT id,
       account_id,
       occurred_at,
       total_amt_usd,
       NTILE(100) OVER (PARTITION BY account_id ORDER BY total_amt_usd) AS total_percentile
FROM orders
ORDER BY account_id `

However, if you wanted to see percentiles across all orders, do this:
`SELECT id,
       account_id,
       occurred_at,
       total_amt_usd,
       NTILE(100) OVER (PARTITION BY account_id ORDER BY total_amt_usd) AS total_percentile
FROM orders
ORDER BY account_id `